In [ ]:
import pandas as pd
import numpy as np
import os
from numba import jit
import json
from tqdm.notebook import tqdm
from datetime import datetime
from numba import jit
import feather

In [ ]:
tqdm.pandas()

# Load dataset

In [ ]:
df = None
for i in tqdm(range(1,5,1)):
    #Load CSV
    _df = pd.read_csv(os.path.join("F:/", "minutesaggregatedsteps_22_01_14", "minutesaggregatedsteps_%s.csv"%str(i)), usecols=['PartitionKey', 'DeviceType', 'DeviceUuid', 'EndTime', 'Model', 'OtherSource', 'ProgramVersion', 'StartTime', 'Steps'], low_memory=False)
    if df is None:
        df = _df
    else:
        df = pd.concat([df,_df])

# Load keyfiles

In [ ]:
df_key = pd.read_csv(os.path.join("F:/", "nyckelfil.csv"), sep = ';')
df_key['DeviceUuId'] = df_key['DeviceUuId'].str.lower()
df_key['ParticipantUuId'] = df_key['ParticipantUuId'].str.lower()
df_key.head()

# Merge ParticipantUuID

In [ ]:
df_merged = df.merge(df_key[['DeviceUuId', 'ParticipantUuId']], left_on = 'DeviceUuid', right_on = 'DeviceUuId', how = 'left')
df_merged = df_merged[~df_merged['ParticipantUuId'].isna()]
df_merged = df_merged.drop('DeviceUuId', axis = 1)
df_merged

# Delete entities whose start time and end time not recorded in the same day 

In [ ]:
def calcualte_time_difference(row):
    start_date = row['StartTime'][:10]
    end_date = row['EndTime'][:10]
    return start_date == end_date
is_same_date = df_merged.progress_apply(calcualte_time_difference, axis = 1)

In [ ]:
df_merged_same_date = df_merged[is_same_date]
df_merged_same_date

# Drop duplicates

In [ ]:
df_merged_same_date_no_dup = df_merged_same_date.drop_duplicates(subset = ['ParticipantUuId', 'StartTime', 'EndTime', 'Steps'])

In [ ]:
df_merged_same_date_no_dup.head()

# Delete outlier

In [ ]:
def check_frequency(row):
    startTime_datetime64 = np.datetime64(row['StartTime'])
    endTime_datetime64 = np.datetime64(row['EndTime'])
    time_difference = (endTime_datetime64 - startTime_datetime64)/np.timedelta64(1, 's')
    if time_difference == 0.0:
        return False
    elif row['Steps']/time_difference >= 5.0:
        return False
    else:
        return True
check_frequency_result = df_merged_same_date_no_dup.progress_apply(check_frequency, axis = 1)

In [ ]:
df_merged_same_date_no_dup = df_merged_same_date_no_dup[check_frequency_result]

# Groupby date and device

In [ ]:
df_merged_same_date_no_dup_groupped = df_merged_same_date_no_dup.groupby(['ParticipantUuId', 'DeviceType', 'PartitionKey'])['Steps'].sum().reset_index()

# Include only one device

In [ ]:
df_merged_same_date_no_dup_groupped_one_device = df_merged_same_date_no_dup_groupped.sort_values(by = ['ParticipantUuId', 'PartitionKey','Steps']).drop_duplicates(subset = ['ParticipantUuId', 'PartitionKey'], keep = 'last')
df_merged_same_date_no_dup_groupped_one_device

In [ ]:
df_daily_aggregated = df_merged_same_date_no_dup_groupped_one_device.copy()

In [ ]:
result = {}
for participant_id in tqdm(df_daily_aggregated['ParticipantUuId'].unique()):
    result[participant_id] = {}
    _df = df_daily_aggregated[df_daily_aggregated['ParticipantUuId'] == participant_id]
    len_2019 = len(_df[(_df['PartitionKey']<=20191231)&(_df['PartitionKey']>=20190101)])
    len_2020 = len(_df[(_df['PartitionKey']<=20201231)&(_df['PartitionKey']>=20200101)])
    len_2021 = len(_df[(_df['PartitionKey']<=20211231)&(_df['PartitionKey']>=20210101)])
    result[participant_id]['2019'] = len_2019
    result[participant_id]['2020'] = len_2020
    result[participant_id]['2021'] = len_2021
result

In [ ]:
result_df = pd.DataFrame(result).transpose().reset_index().rename(columns = {'index': 'ParticipantUuid'})
list_of_participants = result_df[(result_df['2019'] >= 365*0.8) & (result_df['2020'] >= 365*0.8) & (result_df['2021'] >= 365*0.5)]['ParticipantUuid']
df_daily_aggregated[df_daily_aggregated['ParticipantUuid'].isin(list_of_participants)]